In [2]:
import pandas as pd
import requests
import re
import lxml
from bs4 import BeautifulSoup

### User Agent 를 이용하여 200 코드 받기

-----------------------------

브라우저가 많아지고 호환성의 문제가 발생하기 때문에 호환성이 대표적으로 좋은편인 **'Firefox'**, **'Chrome'** 중 Firefox를 선택

In [4]:
chart_url = 'https://music.bugs.co.kr/chart'

headers = {
    'User_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:10.0) Gecko/20100101 Firefox/10.0'
}

In [3]:
res = requests.get(chart_url, headers=headers)
res

<Response [200]>

In [4]:
html = res.text

In [5]:
soup = BeautifulSoup(html, 'html.parser')

### pandas

**pandas** 의 자료구조인 `data_frame`과 `series` 로 크롤링 해보기

----------------------

- data_frame 은 2차적 데이터 구조

- series 는 1차적 데이터 구조


- `requests` 모듈을 사용하여 chrart_url을 가져오게 되면 HTTP Response 의 응답을 받게 된다. Response 200을 받으면 데이터 파싱이 가능한 상태

```python
> res = requests.get(chart_url, headers)
> res
# output <Response [200]>
```

In [5]:
res = requests.get(chart_url, headers)
html = res.text

In [7]:
type(pd.read_html(html))

list

In [8]:
len(pd.read_html(html))

3

In [20]:
chart_df = pd.read_html(html)[0] # 총 3개의 리스트 중에서 첫번째[0] 의 리스트 값을 추출
chart_df

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
1,NaN,2 0변동없음,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
2,NaN,3 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
3,NaN,4 0변동없음,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
4,NaN,5 1 계단 상승,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
5,NaN,6 1 계단 하락,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
6,NaN,7 0변동없음,NaN,곡정보,시간이 들겠지 (Feat. Colde),로꼬,시간이 들겠지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
7,NaN,8 0변동없음,NaN,곡정보,가을 타나 봐,바이브(Vibe),가을 타나 봐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능
8,NaN,9 0변동없음,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능
9,NaN,10 1 계단 상승,NaN,곡정보,하루도 그대를 사랑하지 않은 적이 없었다,임창정,하루도 그대를 사랑하지 않은 적이 없었다,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능


### 크롤링한 데이터프레임의 행과 열을 파악하는 방법( .shape)

In [10]:
chart_df.shape # 100개의 row 와 13개의 columns 로 이루어져있는 것을 확인

(100, 13)

### columns 를 이용하여 데이터프레임의 열에 속한 필드를 알아보기(.columns)

In [11]:
chart_df.columns

Index(['Unnamed: 0', '순위', 'Unnamed: 2', 'Unnamed: 3', '곡', '아티스트', '앨범', '듣기',
       '재생목록', '내앨범', '다운', '영상', '기타'],
      dtype='object')

### set_index 를 이용하여 다시 정렬하기

내가 원하는 레코드의 값만을 지정해서 추출하고 싶을 때가 있는데, `columns=[]` 를 이용하여 추출하고자 하는 필드명을 지정하고, `set_index()`로 기준을 잡아 다시 출력할 수 있다.

-------------------------


In [12]:
pd.DataFrame(chart_df, columns=['순위','곡', '아티스트', '앨범']).set_index('곡')

,순위,아티스트,앨범
곡,,,
삐삐,1 0변동없음,아이유(IU),삐삐
Save (Feat. 팔로알토)(Prod. 코드 쿤스트),2 0변동없음,루피(Loopy),쇼미더머니 777 Episode 3
멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),3 0변동없음,Zion.T,ZZZ
고백,4 0변동없음,양다일,고백
Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),5 2 계단 상승,pH-1 pH-1,쇼미더머니 777 Episode 1
Kiss and Make Up,6 1 계단 하락,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition]
꿈처럼 내린,7 1 계단 상승,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3
넌 (none),8 2 계단 하락,크러쉬(Crush),넌 (none)
시간이 들겠지 (Feat. Colde),9 0변동없음,로꼬,시간이 들겠지


### 데이터프레임에 필드를 추가하고 head, tail 을 사용해보기

- `.head()` 의 기본 추출은 상위 5개의 필드값만 나오며, 10개만 추출하고 싶다면 .head(10)과 같은 옵션을 설정

- `.tail()` 역시 기본 추출은 하위 5개의 필드값이며, 10개만 추출하고 싶다면 .tail(10) 옵션을 설정

In [13]:
chart_df['좋아요'] = 100 # 최초에 좋아요 레코드값을 100으로 통일하여 설정

In [26]:
# 정렬 기준에 따라서 레코드값을 1 ~ 101까지 설정
# 만약 데이터프레임과 설정하려는 레코드값이 다를 경우에는 Length 에러가 발생
## chart_df['좋아요'] = range(1, 30) <-- Length 에러

chart_df['좋아요'] = range(1, 101)

In [24]:
chart_df.head(10)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,4
4,NaN,5 2 계단 상승,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,5
5,NaN,6 1 계단 하락,NaN,곡정보,Kiss and Make Up,Dua Lipa(두아 리파) Dua Lipa(두아 리파),Dua Lipa [Complete Edition],듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,6
6,NaN,7 1 계단 상승,NaN,곡정보,꿈처럼 내린,다비치,뷰티 인사이드 (JTBC 월화드라마) OST - Part.3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,7
7,NaN,8 2 계단 하락,NaN,곡정보,넌 (none),크러쉬(Crush),넌 (none),듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,8
8,NaN,9 0변동없음,NaN,곡정보,시간이 들겠지 (Feat. Colde),로꼬,시간이 들겠지,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,9
9,NaN,10 0변동없음,NaN,곡정보,가을 타나 봐,바이브(Vibe),가을 타나 봐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,10


> ### .head() 옵션

**head 옵션을 사용하면 상위의 지정한 갯수만큼의 필드값을 추출할 수 있다.**

- chart_df.head() # 기본은 5개

- chart_df.head(50) # 50개의 필드 추출

---------------------------

In [16]:
chart_df.head()

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
0,NaN,1 0변동없음,NaN,곡정보,삐삐,아이유(IU),삐삐,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,1
1,NaN,2 0변동없음,NaN,곡정보,Save (Feat. 팔로알토)(Prod. 코드 쿤스트),루피(Loopy),쇼미더머니 777 Episode 3,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,2
2,NaN,3 0변동없음,NaN,곡정보,멋지게 인사하는 법 (Feat. 슬기 of Red Velvet),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,3
3,NaN,4 0변동없음,NaN,곡정보,고백,양다일,고백,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,4
4,NaN,5 2 계단 상승,NaN,곡정보,Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트),pH-1 pH-1,쇼미더머니 777 Episode 1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,5


> ### .tail() 옵션

**tail 옵션을 사용하면 하위의 지정한 갯수만큼의 필드값을 추출할 수 있다.**

- chart_df.tail() # 기본은 5개

- chart_df.tail(50) # 50개의 필드 추출

---------------------------

In [17]:
chart_df.tail(7)

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
93,NaN,94 7 계단 상승,NaN,곡정보,폰서트,10cm(십센치),4.0,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,94
94,NaN,95 17 계단 상승,NaN,곡정보,BAAM,모모랜드(MOMOLAND),Fun to The World,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,95
95,NaN,96 8 계단 하락,NaN,곡정보,DNA,방탄소년단,LOVE YOURSELF 承 'Her',듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,96
96,NaN,97 3 계단 상승,NaN,곡정보,Chandelier,Sia(시아),1000 Forms Of Fear,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,97
97,NaN,98 3 계단 하락,NaN,곡정보,I'm Fine,방탄소년단,LOVE YOURSELF 結 ‘Answer’,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,98
98,NaN,99 0변동없음,NaN,곡정보,밤 (Time for the moon night),여자친구(GFRIEND),여자친구 The 6th Mini Album 'Time for the moon night',듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,99
99,NaN,100 6 계단 상승,NaN,곡정보,뿜뿜,모모랜드(MOMOLAND),GREAT!,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,100


### 레코드값을 이용하여 오름차순, 내림차순하기 `(ascending=True, False)`

-------------------------------

In [36]:
a = chart_df.sort_values('아티스트', ascending=False)
a[['곡', '아티스트']]

,곡,아티스트
12,어찌 잊으오,황치열
78,행복해야해 (Please...),황치열
88,LOVE BOMB,프로미스나인
52,비,폴킴(Paul Kim)
14,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴(Paul Kim)
26,헤어지는 중,펀치 (Punch)
73,Page 0,태연 (TAEYEON) 태연 (TAEYEON)
7,넌 (none),크러쉬(Crush)
27,벚꽃연가,첸 (CHEN)
77,Love U,청하


### 좋아요 기준 50개를 넘는 레코드값만 출력하기

----------------------

In [19]:
mask = chart_df['좋아요'] > 50
chart_df[mask]

,Unnamed: 0,순위,Unnamed: 2,Unnamed: 3,곡,아티스트,앨범,듣기,재생목록,내앨범,다운,영상,기타,좋아요
50,NaN,51 7 계단 상승,NaN,곡정보,너나 해 (Egotistic),마마무(Mamamoo),RED MOON,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,51
51,NaN,52 24 계단 상승,NaN,곡정보,잠꼬대 (Feat. 오혁),Zion.T,ZZZ,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,52
52,NaN,53 4 계단 하락,NaN,곡정보,비,폴킴(Paul Kim),비,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,53
53,NaN,54 1 계단 상승,NaN,곡정보,그때 헤어지면 돼,로이킴,그때 헤어지면 돼,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,54
54,NaN,55 4 계단 상승,NaN,곡정보,여행,볼빨간사춘기,Red Diary Page.2,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,55
55,NaN,56 4 계단 하락,NaN,곡정보,Forever Young,BLACKPINK,SQUARE UP,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,56
56,NaN,57 6 계단 하락,NaN,곡정보,Perfect,10cm(십센치),연애플레이리스트3 (웹드라마) OST - part.1,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,57
57,NaN,58 4 계단 상승,NaN,곡정보,"셋 셀테니 (1, 2, 3!)",승리,THE GREAT SEUNGRI,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,58
58,NaN,59 3 계단 하락,NaN,곡정보,지나오다,닐로(Nil_O),About You,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생 불가,기타 기능,59
59,NaN,60 1 계단 상승,NaN,곡정보,1도 없어,Apink (에이핑크),ONE & SIX,듣기,재생목록에 추가,내 앨범에 담기,flac 다운로드,영상 재생,기타 기능,60


0                                        삐삐
1           Save (Feat. 팔로알토)(Prod. 코드 쿤스트)
2       멋지게 인사하는 법 (Feat. 슬기 of Red Velvet)
3                          Kiss and Make Up
4                                        고백
5      Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트)
6                     시간이 들겠지 (Feat. Colde)
7                                   가을 타나 봐
8                                    꿈처럼 내린
9                    하루도 그대를 사랑하지 않은 적이 없었다
10                                 넌 (none)
11                                   어찌 잊으오
12                                       물어
13                               사이렌(Siren)
14                       이별길 (GOODBYE ROAD)
15    모든 날, 모든 순간 (Every day, Every Moment)
16                            Way Back Home
17      Hate You (Feat. 우원재) (Prod. 코드 쿤스트)
18                                  우리 그만하자
19                                Wow Thing
20                                     벚꽃연가
21            Balance (Prod. By CODE KUNST)
22                              

In [24]:
chart_df['곡']

0                                        삐삐
1           Save (Feat. 팔로알토)(Prod. 코드 쿤스트)
2       멋지게 인사하는 법 (Feat. 슬기 of Red Velvet)
3                          Kiss and Make Up
4                                        고백
5      Good Day (Feat. 팔로알토) (Prod. 코드 쿤스트)
6                     시간이 들겠지 (Feat. Colde)
7                                   가을 타나 봐
8                                    꿈처럼 내린
9                    하루도 그대를 사랑하지 않은 적이 없었다
10                                 넌 (none)
11                                   어찌 잊으오
12                                       물어
13                               사이렌(Siren)
14                       이별길 (GOODBYE ROAD)
15    모든 날, 모든 순간 (Every day, Every Moment)
16                            Way Back Home
17      Hate You (Feat. 우원재) (Prod. 코드 쿤스트)
18                                  우리 그만하자
19                                Wow Thing
20                                     벚꽃연가
21            Balance (Prod. By CODE KUNST)
22                              